This notebook migrates test data from t_* tables to g_* tables,

then port  `app_poc.py` to `app_g.py`

In [1]:
import pandas as pd 
from db import *
from config_poc import *

pd.set_option('display.max_columns', None)

In [7]:
sql_stmt = """
select  c.table_name, c.column_name --, c.data_type,  c.*
from information_schema.columns c
where 1=1
--and c.table_catalog like 'faculty-Cornell-CS%'
and c.table_schema = 'main'
and c.table_name like 'g_%'
order by c.table_catalog,c.table_schema,c.table_name, c.ordinal_position
;
"""

In [8]:
with DBConn() as _conn:
    df = pd.read_sql(sql_stmt, _conn)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\496018617.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_stmt, _conn)


In [9]:
df

,table_name,column_name
0,g_column_props,table_name
1,g_column_props,col_name
2,g_column_props,is_system_col
3,g_column_props,is_user_key
4,g_column_props,is_required
...,...,...
64,g_work,url
65,g_work,note
66,g_work,summary
67,g_work,authors


In [10]:
df.to_csv("g_table_columns.csv", index=False)

In [13]:
uid = '0'

## g_note

In [5]:
table_name = "g_note"
src_table_name = table_name.replace("g_", "t_")

In [6]:
with DBConn() as _conn:
    df = pd.read_sql(f"""
        select * from {src_table_name};
    """, _conn)
df 

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\108093257.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"""


,name,note,url,tags,ts,id,ref_type,ref_key
0,Campbell Research Group,Our group works on algorithms and experimental...,https://campbell.mae.cornell.edu/,robotics autonomy,2023-04-08 22:02:22.899622,c1b08a22-80bf-4869-99d5-cc4656dde20d,None,None
1,Cornell AI Group,this group has over 50 faculties,https://www.cs.cornell.edu/research/ai,AI,2023-04-08 18:41:24.944832,531160e6-bdf1-40e3-ab11-8307d4b2f95b,None,None
2,Kevin Tang Home page,I teach in the School of ECE (Electrical and C...,http://people.ece.cornell.edu/atang/,"Network, AI",2023-04-20 22:27:15.156655,t_faculty # http://people.ece.cornell.edu/atang/,t_person,http://people.ece.cornell.edu/atang/
3,AI is eating the world,None,https://www.forbes.com/sites/cognitiveworld/20...,AI,2023-04-20 22:29:07.651777,d3bf60a0-4445-4662-927a-c965de9227be,None,None
4,"Building Human Intelligence at Scale, to Save ...","Po-Shen Loh is a math professor, researcher, a...",https://www.youtube.com/watch?v=oO-akX66i24,"AI, education",None,9b72a022-bea2-4658-9be2-491565a6bb1b,None,None


In [11]:
cols_str = """
id
uid
ts
name
url
note
tags
ref_key
ref_val
"""
cols = [c.strip() for c in cols_str.split("\n") if c.strip()]
cols 

['id', 'uid', 'ts', 'name', 'url', 'note', 'tags', 'ref_key', 'ref_val']

In [15]:
print("\n, ".join(cols))

id
, uid
, ts
, name
, url
, note
, tags
, ref_key
, ref_val


In [17]:
col_lst = "\n, ".join(cols)

In [23]:
insert_sql = f"""
insert into {table_name} ({col_lst})
select uuid()
    , '{uid}'
    , now()
    , name
    , url
    , note
    , tags
    , case 
        when ref_type is NULL then NULL
        else 'g_person#url'
      end
    , ref_key
from {src_table_name}
;
"""

In [24]:
print(insert_sql)


insert into g_note (id
, uid
, ts
, name
, url
, note
, tags
, ref_key
, ref_val)
select uuid()
    , '0'
    , now()
    , name
    , url
    , note
    , tags
    , case 
        when ref_type is NULL then NULL
        else 'g_person#url'
      end
    , ref_key
from t_note
;



In [25]:
with DBConn() as _conn:
    df_test = pd.read_sql(f"""
select uuid()
    , '0'
    , now()
    , name
    , url
    , note
    , tags
    , case 
        when ref_type is NULL then NULL
        else 'g_person#url'
      end
    , ref_key
from t_note
    """, _conn)
df_test

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\3845250933.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql(f"""


,uuid(),'0',now(),name,url,note,tags,CASE WHEN ((ref_type IS NULL)) THEN (NULL) ELSE 'g_person#url' END,ref_key
0,54aa40bf-e646-4a2d-ab03-c0d2e6462dd4,0,2023-04-26 02:22:03.839,Campbell Research Group,https://campbell.mae.cornell.edu/,Our group works on algorithms and experimental...,robotics autonomy,None,None
1,dd4096ab-bb15-4a73-a43d-689337aec274,0,2023-04-26 02:22:03.839,Cornell AI Group,https://www.cs.cornell.edu/research/ai,this group has over 50 faculties,AI,None,None
2,a1cbe99c-d788-4d2c-9fba-240e3aff09f1,0,2023-04-26 02:22:03.839,Kevin Tang Home page,http://people.ece.cornell.edu/atang/,I teach in the School of ECE (Electrical and C...,"Network, AI",g_person#url,http://people.ece.cornell.edu/atang/
3,60f47627-d7c8-4771-b70a-a38a9d676966,0,2023-04-26 02:22:03.839,AI is eating the world,https://www.forbes.com/sites/cognitiveworld/20...,None,AI,None,None
4,54e437cd-d377-4ebb-9876-65957d23c798,0,2023-04-26 02:22:03.839,"Building Human Intelligence at Scale, to Save ...",https://www.youtube.com/watch?v=oO-akX66i24,"Po-Shen Loh is a math professor, researcher, a...","AI, education",None,None


Looks good

In [26]:
# insert
with DBConn() as _conn:
    _conn.execute(insert_sql)
    _conn.commit()    

In [27]:
# query
with DBConn() as _conn:
    df_test = pd.read_sql(f"select * from {table_name};", _conn)
df_test    

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\1064854284.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql(f"select * from {table_name};", _conn)


,id,uid,ts,name,url,note,tags,ref_key,ref_val
0,d4f3a966-4d8b-4b8c-81b5-42dae8d459da,0,2023-04-25 22:23:47.173-04,Campbell Research Group,https://campbell.mae.cornell.edu/,Our group works on algorithms and experimental...,robotics autonomy,None,None
1,cd7ba169-075b-4611-96da-8330beed0ddd,0,2023-04-25 22:23:47.173-04,Cornell AI Group,https://www.cs.cornell.edu/research/ai,this group has over 50 faculties,AI,None,None
2,066179d7-712b-4def-bdbe-053ad509b368,0,2023-04-25 22:23:47.173-04,Kevin Tang Home page,http://people.ece.cornell.edu/atang/,I teach in the School of ECE (Electrical and C...,"Network, AI",g_person#url,http://people.ece.cornell.edu/atang/
3,b030a4f5-8124-4360-842a-8a99366acb8c,0,2023-04-25 22:23:47.173-04,AI is eating the world,https://www.forbes.com/sites/cognitiveworld/20...,None,AI,None,None
4,391c7462-eeee-43a3-8518-f09df1aab21c,0,2023-04-25 22:23:47.173-04,"Building Human Intelligence at Scale, to Save ...",https://www.youtube.com/watch?v=oO-akX66i24,"Po-Shen Loh is a math professor, researcher, a...","AI, education",None,None


## g_work

In [29]:
table_name = "g_work"
src_table_name = table_name.replace("g_", "t_")

In [30]:
with DBConn() as _conn:
    df_test = pd.read_sql(f"""
        select * from {src_table_name};
    """, _conn)
df_test

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\2841525488.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql(f"""


,type,name,url,summary,authors,tags,note,ts_created,id,ts
0,publication,Task-aware Privacy Preservation for Multi-dime...,http://people.ece.cornell.edu/atang/pub/22/ICM...,Extensive experiments demonstrate that our tas...,"J. Cheng, A. Tang and S. Chinchali",None,None,None,atang/pub/22/ICML2022.pdf,None
1,publication,Data Sharing and Compression for Cooperative N...,http://people.ece.cornell.edu/atang/pub/21/Neu...,we present theoretical compression results for...,"J. Cheng, M. Pavone, S. Katti, S. Chinchali, a...",None,None,None,atang/pub/21/NeurIPS-2021.pdf,None
2,publication,On upper bounding Shannon capacity of graph th...,http://people.ece.cornell.edu/atang/pub/19/opt...,"In this paper, we consider the\npossibility o...","Yingjie Bi, Ao Tan",None,None,None,b62cf974-c5e4-494a-a29c-e34576bb1ec2,None
3,publication,Routing Stability in Hybrid Software-Defined N...,http://people.ece.cornell.edu/atang/pub/19/ToN...,The design is implemented and extensively tes...,"S. Tseng, A. Tang, G. Choudhury and S. Tse",None,test,None,6456ab1c-84e3-4207-b39c-f24937654d82,None


In [31]:
cols_str = """
id
uid
ts
work_type
name
url
note
summary
authors
tags

"""
cols = [c.strip() for c in cols_str.split("\n") if c.strip()]
cols 

['id',
 'uid',
 'ts',
 'work_type',
 'name',
 'url',
 'note',
 'summary',
 'authors',
 'tags']

In [34]:
col_lst = "\n, ".join(cols)
print(col_lst)

id
, uid
, ts
, work_type
, name
, url
, note
, summary
, authors
, tags


In [35]:
df_test.columns

Index(['type', 'name', 'url', 'summary', 'authors', 'tags', 'note',
       'ts_created', 'id', 'ts'],
      dtype='object')

In [36]:
insert_sql = f"""
insert into {table_name} ({col_lst})
select id
    , '{uid}'
    , now()
    , type
    , name
    , url
    , note
    , summary
    , authors
    , tags
from {src_table_name}
;
"""

In [37]:
print(insert_sql)


insert into g_work (id
, uid
, ts
, work_type
, name
, url
, note
, summary
, authors
, tags)
select id
    , '0'
    , now()
    , type
    , name
    , url
    , note
    , summary
    , authors
    , tags
from t_work
;



In [38]:
with DBConn() as _conn:
    df_test2 = pd.read_sql(f"""
select id
    , '0'
    , now()
    , type
    , name
    , url
    , note
    , summary
    , authors
    , tags
from t_work
    """, _conn)
df_test2

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\2607157703.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test2 = pd.read_sql(f"""


,id,'0',now(),type,name,url,note,summary,authors,tags
0,atang/pub/22/ICML2022.pdf,0,2023-04-26 02:38:25.947,publication,Task-aware Privacy Preservation for Multi-dime...,http://people.ece.cornell.edu/atang/pub/22/ICM...,None,Extensive experiments demonstrate that our tas...,"J. Cheng, A. Tang and S. Chinchali",None
1,atang/pub/21/NeurIPS-2021.pdf,0,2023-04-26 02:38:25.947,publication,Data Sharing and Compression for Cooperative N...,http://people.ece.cornell.edu/atang/pub/21/Neu...,None,we present theoretical compression results for...,"J. Cheng, M. Pavone, S. Katti, S. Chinchali, a...",None
2,b62cf974-c5e4-494a-a29c-e34576bb1ec2,0,2023-04-26 02:38:25.947,publication,On upper bounding Shannon capacity of graph th...,http://people.ece.cornell.edu/atang/pub/19/opt...,None,"In this paper, we consider the\npossibility o...","Yingjie Bi, Ao Tan",None
3,6456ab1c-84e3-4207-b39c-f24937654d82,0,2023-04-26 02:38:25.947,publication,Routing Stability in Hybrid Software-Defined N...,http://people.ece.cornell.edu/atang/pub/19/ToN...,test,The design is implemented and extensively tes...,"S. Tseng, A. Tang, G. Choudhury and S. Tse",None


In [39]:
# insert
with DBConn() as _conn:
    _conn.execute(insert_sql)
    _conn.commit()   

In [40]:
# query
with DBConn() as _conn:
    df_test3 = pd.read_sql(f"select * from {table_name};", _conn)
df_test3

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\1942543153.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test3 = pd.read_sql(f"select * from {table_name};", _conn)


,id,uid,ts,work_type,name,url,note,summary,authors,tags
0,atang/pub/22/ICML2022.pdf,0,2023-04-25 22:39:21.303-04,publication,Task-aware Privacy Preservation for Multi-dime...,http://people.ece.cornell.edu/atang/pub/22/ICM...,None,Extensive experiments demonstrate that our tas...,"J. Cheng, A. Tang and S. Chinchali",None
1,atang/pub/21/NeurIPS-2021.pdf,0,2023-04-25 22:39:21.303-04,publication,Data Sharing and Compression for Cooperative N...,http://people.ece.cornell.edu/atang/pub/21/Neu...,None,we present theoretical compression results for...,"J. Cheng, M. Pavone, S. Katti, S. Chinchali, a...",None
2,b62cf974-c5e4-494a-a29c-e34576bb1ec2,0,2023-04-25 22:39:21.303-04,publication,On upper bounding Shannon capacity of graph th...,http://people.ece.cornell.edu/atang/pub/19/opt...,None,"In this paper, we consider the\npossibility o...","Yingjie Bi, Ao Tan",None
3,6456ab1c-84e3-4207-b39c-f24937654d82,0,2023-04-25 22:39:21.303-04,publication,Routing Stability in Hybrid Software-Defined N...,http://people.ece.cornell.edu/atang/pub/19/ToN...,test,The design is implemented and extensively tes...,"S. Tseng, A. Tang, G. Choudhury and S. Tse",None


## g_person

In [41]:
table_name = "g_person"
src_table_name = table_name.replace("g_", "t_")

In [ ]:
with DBConn() as _conn:
    df_src = pd.read_sql(f"""
        select * from {src_table_name};
    """, _conn)

In [43]:
df_src.head(3)

,name,url,email,first_name,mid_name,last_name,note,id,ts,job_title,person_type,phd_univ,phd_year,research_area,research_concentration,research_focus,img_url,phone,cell_phone,office_address,department,school,org
0,Yuchong Geng,https://yuchong-geng.github.io/,yg534@cornell.edu,Yuchong,None,Geng,None,yg534@cornell.edu,None,None,student,None,None,AI Network System,None,None,None,None,None,None,None,None,None
1,Faraz Farahvash,http://networks.ece.cornell.edu/faraz,ff227@cornell.edu,Faraz,None,Farahvash,None,ff227@cornell.edu,None,None,student,None,None,None,None,None,None,None,None,None,None,None,None
2,Mohamed Abdelfattah,https://www.mohsaied.com,,None,None,None,None,409a467e-159e-411e-aedf-206e0ce0d71e,2023-04-23 14:48:37.555-04,Assistant Professor,faculty,Univ Toronto,2016,"Artificial Intelligence, Machine Learning",Artificial Intelligence,,https://www.cs.cornell.edu/sites/default/files...,,,,"Electrical and Computer Engineering, Cornell T...",,None


In [44]:
cols_str = """
id
uid
ts
person_type
name
url
note
email
first_name
mid_name
last_name
job_title
phd_univ
phd_year
research_area
research_concentration
research_focus
img_url
phone
cell_phone
office_address
department
org

"""
cols = [c.strip() for c in cols_str.split("\n") if c.strip()]
cols 

['id',
 'uid',
 'ts',
 'person_type',
 'name',
 'url',
 'note',
 'email',
 'first_name',
 'mid_name',
 'last_name',
 'job_title',
 'phd_univ',
 'phd_year',
 'research_area',
 'research_concentration',
 'research_focus',
 'img_url',
 'phone',
 'cell_phone',
 'office_address',
 'department',
 'org']

In [45]:
col_lst = "\n, ".join(cols)
print(col_lst)

id
, uid
, ts
, person_type
, name
, url
, note
, email
, first_name
, mid_name
, last_name
, job_title
, phd_univ
, phd_year
, research_area
, research_concentration
, research_focus
, img_url
, phone
, cell_phone
, office_address
, department
, org


In [46]:
df_src.columns

Index(['name', 'url', 'email', 'first_name', 'mid_name', 'last_name', 'note',
       'id', 'ts', 'job_title', 'person_type', 'phd_univ', 'phd_year',
       'research_area', 'research_concentration', 'research_focus', 'img_url',
       'phone', 'cell_phone', 'office_address', 'department', 'school', 'org'],
      dtype='object')

In [49]:
insert_sql = f"""
insert into {table_name} ({col_lst})
select id
    , '{uid}'
    , now()
    , person_type
    , name
    , url
    , note
    , email
    , first_name
    , mid_name
    , last_name
    , job_title
    , phd_univ
    , phd_year
    , research_area
    , research_concentration
    , research_focus
    , img_url
    , phone
    , cell_phone
    , office_address
    , department
    , 'Cornell Univ'
from {src_table_name}
;
"""
print(insert_sql)


insert into g_person (id
, uid
, ts
, person_type
, name
, url
, note
, email
, first_name
, mid_name
, last_name
, job_title
, phd_univ
, phd_year
, research_area
, research_concentration
, research_focus
, img_url
, phone
, cell_phone
, office_address
, department
, org)
select id
    , '0'
    , now()
    , person_type
    , name
    , url
    , note
    , email
    , first_name
    , mid_name
    , last_name
    , job_title
    , phd_univ
    , phd_year
    , research_area
    , research_concentration
    , research_focus
    , img_url
    , phone
    , cell_phone
    , office_address
    , department
    , 'Cornell Univ'
from t_person
;



In [50]:
with DBConn() as _conn:
    df_test2 = pd.read_sql(f"""
select id
    , '0'
    , now()
    , person_type
    , name
    , url
    , note
    , email
    , first_name
    , mid_name
    , last_name
    , job_title
    , phd_univ
    , phd_year
    , research_area
    , research_concentration
    , research_focus
    , img_url
    , phone
    , cell_phone
    , office_address
    , department
    , 'Cornell Univ'
from t_person
    """, _conn)
df_test2.head(5)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\893381946.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test2 = pd.read_sql(f"""


,id,'0',now(),person_type,name,url,note,email,first_name,mid_name,last_name,job_title,phd_univ,phd_year,research_area,research_concentration,research_focus,img_url,phone,cell_phone,office_address,department,'Cornell Univ'
0,yg534@cornell.edu,0,2023-04-26 02:52:17.095,student,Yuchong Geng,https://yuchong-geng.github.io/,None,yg534@cornell.edu,Yuchong,None,Geng,None,None,None,AI Network System,None,None,None,None,None,None,None,Cornell Univ
1,ff227@cornell.edu,0,2023-04-26 02:52:17.095,student,Faraz Farahvash,http://networks.ece.cornell.edu/faraz,None,ff227@cornell.edu,Faraz,None,Farahvash,None,None,None,None,None,None,None,None,None,None,None,Cornell Univ
2,409a467e-159e-411e-aedf-206e0ce0d71e,0,2023-04-26 02:52:17.095,faculty,Mohamed Abdelfattah,https://www.mohsaied.com,None,,None,None,None,Assistant Professor,Univ Toronto,2016,"Artificial Intelligence, Machine Learning",Artificial Intelligence,,https://www.cs.cornell.edu/sites/default/files...,,,,"Electrical and Computer Engineering, Cornell T...",Cornell Univ
3,23605352-f553-4289-86fd-e967a3ae693b,0,2023-04-26 02:52:17.095,faculty,Jayadev Acharya,https://people.ece.cornell.edu/acharya/,None,,None,None,None,Assistant Professor,Univ California San Diego,2014,"Artificial Intelligence, Theory of Computing","Artificial Intelligence, Theory of Computation","Information theory, machine learning, and algo...",https://www.cs.cornell.edu/sites/default/files...,,,,"Electrical and Computer Engineering, CS Field ...",Cornell Univ
4,edf90ce7-67ff-474d-8b17-47f6e43fa5c2,0,2023-04-26 02:52:17.095,faculty,Rachit Agarwal,http://www.cs.cornell.edu/~ragarwal/,None,,None,None,None,Associate Professor,Univ Illinois Urbana-Champaign,2013,"Systems and Networking, Theory of Computing","Systems, Theory of Computation","Distributed systems, systems for big data anal...",https://www.cs.cornell.edu/sites/default/files...,,,,"Computer Science, CS Field Member",Cornell Univ


In [51]:
# insert
with DBConn() as _conn:
    _conn.execute(insert_sql)
    _conn.commit()  

In [52]:
# query
with DBConn() as _conn:
    df_test3 = pd.read_sql(f"select * from {table_name};", _conn)
df_test3.head(5)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\199551690.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test3 = pd.read_sql(f"select * from {table_name};", _conn)


,id,uid,ts,person_type,name,url,note,email,first_name,mid_name,last_name,job_title,phd_univ,phd_year,research_area,research_concentration,research_focus,img_url,phone,cell_phone,office_address,department,org
0,yg534@cornell.edu,0,2023-04-25 22:53:12.026-04,student,Yuchong Geng,https://yuchong-geng.github.io/,None,yg534@cornell.edu,Yuchong,None,Geng,None,None,None,AI Network System,None,None,None,None,None,None,None,Cornell Univ
1,ff227@cornell.edu,0,2023-04-25 22:53:12.026-04,student,Faraz Farahvash,http://networks.ece.cornell.edu/faraz,None,ff227@cornell.edu,Faraz,None,Farahvash,None,None,None,None,None,None,None,None,None,None,None,Cornell Univ
2,409a467e-159e-411e-aedf-206e0ce0d71e,0,2023-04-25 22:53:12.026-04,faculty,Mohamed Abdelfattah,https://www.mohsaied.com,None,,None,None,None,Assistant Professor,Univ Toronto,2016,"Artificial Intelligence, Machine Learning",Artificial Intelligence,,https://www.cs.cornell.edu/sites/default/files...,,,,"Electrical and Computer Engineering, Cornell T...",Cornell Univ
3,23605352-f553-4289-86fd-e967a3ae693b,0,2023-04-25 22:53:12.026-04,faculty,Jayadev Acharya,https://people.ece.cornell.edu/acharya/,None,,None,None,None,Assistant Professor,Univ California San Diego,2014,"Artificial Intelligence, Theory of Computing","Artificial Intelligence, Theory of Computation","Information theory, machine learning, and algo...",https://www.cs.cornell.edu/sites/default/files...,,,,"Electrical and Computer Engineering, CS Field ...",Cornell Univ
4,edf90ce7-67ff-474d-8b17-47f6e43fa5c2,0,2023-04-25 22:53:12.026-04,faculty,Rachit Agarwal,http://www.cs.cornell.edu/~ragarwal/,None,,None,None,None,Associate Professor,Univ Illinois Urbana-Champaign,2013,"Systems and Networking, Theory of Computing","Systems, Theory of Computation","Distributed systems, systems for big data anal...",https://www.cs.cornell.edu/sites/default/files...,,,,"Computer Science, CS Field Member",Cornell Univ


## g_entity

### entity_type = `research_group`

In [53]:
table_name = "g_entity"
src_table_name = "t_research_group"

In [54]:
with DBConn() as _conn:
    df_src = pd.read_sql(f"""
        select * from {src_table_name};
    """, _conn)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\298127963.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_src = pd.read_sql(f"""


In [55]:
df_src.head(5)

,name,url,note,id,ts
0,Artificial Intelligence,https://www.cs.cornell.edu/research/ai,None,ccbbb778-7029-4e4b-a166-5ef8cdf21fad,2023-04-23 19:14:58.236-04
1,Machine Learning,https://www.cs.cornell.edu/research/machinelea...,None,409457be-b415-4413-bdff-7f96d06adbcf,2023-04-23 19:14:58.236-04
2,Theory of Computing,https://www.cs.cornell.edu/research/theory,None,e0a7729b-d4f4-4d90-98b3-04d28a07d64f,2023-04-23 19:14:58.236-04
3,Systems and Networking,https://www.cs.cornell.edu/research/systems,None,0156bf23-5aed-46c1-bcd5-8744499152b1,2023-04-23 19:14:58.236-04
4,Computer Architecture & VLSI,https://www.cs.cornell.edu/research/architecture,None,c09c1373-e525-4057-ae50-d2752cdbc42e,2023-04-23 19:14:58.236-04


In [56]:
cols_str = """
id
uid
ts
entity_type
name
url
note

"""
cols = [c.strip() for c in cols_str.split("\n") if c.strip()]
cols 

['id', 'uid', 'ts', 'entity_type', 'name', 'url', 'note']

In [57]:
col_lst = "\n, ".join(cols)
print(col_lst)

id
, uid
, ts
, entity_type
, name
, url
, note


In [58]:
df_src.columns

Index(['name', 'url', 'note', 'id', 'ts'], dtype='object')

In [59]:
insert_sql = f"""
insert into {table_name} ({col_lst})
select id
    , '{uid}'
    , now()
    , 'research_group'
    , name
    , url
    , note
from {src_table_name}
;
"""
print(insert_sql)


insert into g_entity (id
, uid
, ts
, entity_type
, name
, url
, note)
select id
    , '0'
    , now()
    , 'research_group'
    , name
    , url
    , note
from t_research_group
;



In [61]:
with DBConn() as _conn:
    df_test2 = pd.read_sql(f"""
select id
    , '0'
    , now()
    , 'research_group'
    , name
    , url
    , note
from t_research_group
    """, _conn)
df_test2.head(5)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\3826589568.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test2 = pd.read_sql(f"""


,id,'0',now(),'research_group',name,url,note
0,ccbbb778-7029-4e4b-a166-5ef8cdf21fad,0,2023-04-26 03:06:53.539,research_group,Artificial Intelligence,https://www.cs.cornell.edu/research/ai,None
1,409457be-b415-4413-bdff-7f96d06adbcf,0,2023-04-26 03:06:53.539,research_group,Machine Learning,https://www.cs.cornell.edu/research/machinelea...,None
2,e0a7729b-d4f4-4d90-98b3-04d28a07d64f,0,2023-04-26 03:06:53.539,research_group,Theory of Computing,https://www.cs.cornell.edu/research/theory,None
3,0156bf23-5aed-46c1-bcd5-8744499152b1,0,2023-04-26 03:06:53.539,research_group,Systems and Networking,https://www.cs.cornell.edu/research/systems,None
4,c09c1373-e525-4057-ae50-d2752cdbc42e,0,2023-04-26 03:06:53.539,research_group,Computer Architecture & VLSI,https://www.cs.cornell.edu/research/architecture,None


In [62]:
# insert
with DBConn() as _conn:
    _conn.execute(insert_sql)
    _conn.commit()   

In [63]:
# query
with DBConn() as _conn:
    df_test3 = pd.read_sql(f"select * from {table_name};", _conn)
df_test3

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\1942543153.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test3 = pd.read_sql(f"select * from {table_name};", _conn)


,id,uid,ts,entity_type,name,url,note
0,ccbbb778-7029-4e4b-a166-5ef8cdf21fad,0,2023-04-25 23:07:14.609-04,research_group,Artificial Intelligence,https://www.cs.cornell.edu/research/ai,None
1,409457be-b415-4413-bdff-7f96d06adbcf,0,2023-04-25 23:07:14.609-04,research_group,Machine Learning,https://www.cs.cornell.edu/research/machinelea...,None
2,e0a7729b-d4f4-4d90-98b3-04d28a07d64f,0,2023-04-25 23:07:14.609-04,research_group,Theory of Computing,https://www.cs.cornell.edu/research/theory,None
3,0156bf23-5aed-46c1-bcd5-8744499152b1,0,2023-04-25 23:07:14.609-04,research_group,Systems and Networking,https://www.cs.cornell.edu/research/systems,None
4,c09c1373-e525-4057-ae50-d2752cdbc42e,0,2023-04-25 23:07:14.609-04,research_group,Computer Architecture & VLSI,https://www.cs.cornell.edu/research/architecture,None
5,8269530c-07b7-410f-a42b-ca680571e24b,0,2023-04-25 23:07:14.609-04,research_group,Human Interaction,https://www.cs.cornell.edu/research/humaninter...,None
6,04b726d5-7b98-4c39-9beb-9401d56bb716,0,2023-04-25 23:07:14.609-04,research_group,Graphics,https://www.cs.cornell.edu/research/graphics,None
7,62af2b99-c5df-4ba7-abd4-dc9e527e9552,0,2023-04-25 23:07:14.609-04,research_group,Vision,https://www.cs.cornell.edu/research/vision,None
8,1e61712b-fc04-4b51-b42d-7158dfac85fb,0,2023-04-25 23:07:14.609-04,research_group,Robotics,https://robotics.cornell.edu/,None
9,efdc481a-8f6f-4438-b4be-7bd7264a5496,0,2023-04-25 23:07:14.609-04,research_group,Scientific Computing,https://www.cs.cornell.edu/research/scientif,None


### entity_type = `org`

In [65]:
with DBConn() as _conn:
    df_tmp = pd.read_sql(f"""
select uuid()
    , '0'
    , now()
    , 'org'
    , 'Cornell Univ'
    , 'https://www.cornell.edu/'
    , Null
;
    """, _conn)
df_tmp

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\3913033656.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tmp = pd.read_sql(f"""


,uuid(),'0',now(),'org','Cornell Univ','https://www.cornell.edu/',NULL
0,06e2ea82-64ec-47bb-b988-18e71ddb470a,0,2023-04-26 03:14:16.887,org,Cornell Univ,https://www.cornell.edu/,None


In [66]:
insert_sql = f"""
insert into g_entity (id
, uid
, ts
, entity_type
, name
, url
, note)
select uuid()
    , '0'
    , now()
    , 'org'
    , 'Cornell Univ'
    , 'https://www.cornell.edu/'
    , Null
;
;
"""

In [67]:
# insert
with DBConn() as _conn:
    _conn.execute(insert_sql)
    _conn.commit()   

In [68]:
# query
with DBConn() as _conn:
    df_test3 = pd.read_sql(f"select * from g_entity where entity_type='org';", _conn)
df_test3

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\4066688269.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test3 = pd.read_sql(f"select * from g_entity where entity_type='org';", _conn)


,id,uid,ts,entity_type,name,url,note
0,347d84f3-cc06-4b04-b8be-95bfb573c3d1,0,2023-04-25 23:14:54.295-04,org,Cornell Univ,https://www.cornell.edu/,None


## g_relation

### rel_type = `person-work`

In [69]:
table_name = "g_relation"
src_table_name = "t_person_work"

In [70]:
with DBConn() as _conn:
    df_src = pd.read_sql(f"""
        select * from {src_table_name};
    """, _conn)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\298127963.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_src = pd.read_sql(f"""


In [71]:
df_src

,id,ts,ref_type,ref_key,ref_type_2,ref_key_2
0,atang/pub/22/ICML2022.pdf,2023-04-16 10:01:01,t_person,http://people.ece.cornell.edu/atang/,t_work,atang/pub/22/ICML2022.pdf
1,atang/pub/21/NeurIPS-2021.pdf,2023-04-16 10:01:02,t_person,http://people.ece.cornell.edu/atang/,t_work,atang/pub/21/NeurIPS-2021.pdf
2,fff7e2d9-7f5c-47c0-aa37-6f5f4b48757b,2023-04-23 13:08:54.664621,t_person,http://people.ece.cornell.edu/atang/,t_work,b62cf974-c5e4-494a-a29c-e34576bb1ec2


In [72]:
cols_str = """
id
uid
ts
rel_type
ref_key
ref_val
ref_key_sub
ref_val_sub

"""
cols = [c.strip() for c in cols_str.split("\n") if c.strip()]
cols 

['id',
 'uid',
 'ts',
 'rel_type',
 'ref_key',
 'ref_val',
 'ref_key_sub',
 'ref_val_sub']

In [73]:
col_lst = "\n, ".join(cols)
print(col_lst)

id
, uid
, ts
, rel_type
, ref_key
, ref_val
, ref_key_sub
, ref_val_sub


In [74]:
df_src.columns

Index(['id', 'ts', 'ref_type', 'ref_key', 'ref_type_2', 'ref_key_2'], dtype='object')

In [76]:
insert_sql = f"""
insert into {table_name} ({col_lst})
select id
    , '{uid}'
    , ts
    , 'person-work'
, 'g_person#url'
, ref_key
, 'g_work#id'
, ref_key_2
from {src_table_name}
;
"""
print(insert_sql)


insert into g_relation (id
, uid
, ts
, rel_type
, ref_key
, ref_val
, ref_key_sub
, ref_val_sub)
select id
    , '0'
    , ts
    , 'person-work'
, 'g_person#url'
, ref_key
, 'g_work#id'
, ref_key_2
from t_person_work
;



In [77]:
with DBConn() as _conn:
    df_test2 = pd.read_sql(f"""
select id
    , '0'
    , ts
    , 'person-work'
, 'g_person#url'
, ref_key
, 'g_work#id'
, ref_key_2
from t_person_work
    """, _conn)
df_test2

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\765401097.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test2 = pd.read_sql(f"""


,id,'0',ts,'person-work','g_person#url',ref_key,'g_work#id',ref_key_2
0,atang/pub/22/ICML2022.pdf,0,2023-04-16 10:01:01,person-work,g_person#url,http://people.ece.cornell.edu/atang/,g_work#id,atang/pub/22/ICML2022.pdf
1,atang/pub/21/NeurIPS-2021.pdf,0,2023-04-16 10:01:02,person-work,g_person#url,http://people.ece.cornell.edu/atang/,g_work#id,atang/pub/21/NeurIPS-2021.pdf
2,fff7e2d9-7f5c-47c0-aa37-6f5f4b48757b,0,2023-04-23 13:08:54.664621,person-work,g_person#url,http://people.ece.cornell.edu/atang/,g_work#id,b62cf974-c5e4-494a-a29c-e34576bb1ec2


In [78]:
# insert
with DBConn() as _conn:
    _conn.execute(insert_sql)
    _conn.commit()  

In [91]:
# query
with DBConn() as _conn:
    df_test3 = pd.read_sql(f"select * from {table_name} where rel_type='person-work';", _conn)
df_test3

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\2382609479.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test3 = pd.read_sql(f"select * from {table_name} where rel_type='person-work';", _conn)


,id,uid,ts,rel_type,ref_key,ref_val,ref_key_sub,ref_val_sub
0,atang/pub/22/ICML2022.pdf,0,2023-04-16 10:01:01,person-work,g_person#url,http://people.ece.cornell.edu/atang/,g_work#id,atang/pub/22/ICML2022.pdf
1,atang/pub/21/NeurIPS-2021.pdf,0,2023-04-16 10:01:02,person-work,g_person#url,http://people.ece.cornell.edu/atang/,g_work#id,atang/pub/21/NeurIPS-2021.pdf
2,fff7e2d9-7f5c-47c0-aa37-6f5f4b48757b,0,2023-04-23 13:08:54.664621,person-work,g_person#url,http://people.ece.cornell.edu/atang/,g_work#id,b62cf974-c5e4-494a-a29c-e34576bb1ec2


### rel_type = `team-person`

default team by faculty

In [80]:
table_name = "g_relation"
src_table_name = "t_person_team"

In [81]:
with DBConn() as _conn:
    df_src = pd.read_sql(f"""
        select * from {src_table_name};
    """, _conn)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\298127963.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_src = pd.read_sql(f"""


In [82]:
df_src

,id,ts,ref_type,ref_key,ref_type_2,ref_key_2
0,t_person # http://people.ece.cornell.edu/atang/,2023-04-16 10:01:01,t_team,http://networks.ece.cornell.edu/,t_person,http://people.ece.cornell.edu/atang/
1,t_person # https://yuchong-geng.github.io/,2023-04-16 10:01:01,t_team,http://networks.ece.cornell.edu/,t_person,https://yuchong-geng.github.io/
2,t_person # http://networks.ece.cornell.edu/faraz,2023-04-16 10:01:01,t_team,http://networks.ece.cornell.edu/,t_person,http://networks.ece.cornell.edu/faraz


In [83]:
cols_str = """
id
uid
ts
rel_type
ref_key
ref_val
ref_key_sub
ref_val_sub

"""
cols = [c.strip() for c in cols_str.split("\n") if c.strip()]
cols 

['id',
 'uid',
 'ts',
 'rel_type',
 'ref_key',
 'ref_val',
 'ref_key_sub',
 'ref_val_sub']

In [84]:
col_lst = "\n, ".join(cols)
print(col_lst)

id
, uid
, ts
, rel_type
, ref_key
, ref_val
, ref_key_sub
, ref_val_sub


In [85]:
df_src.columns

Index(['id', 'ts', 'ref_type', 'ref_key', 'ref_type_2', 'ref_key_2'], dtype='object')

In [86]:
insert_sql = f"""
insert into {table_name} ({col_lst})
select id
    , '{uid}'
    , ts
    , 'team-person'
, 'g_person#url'
, ref_key
, 'g_person#url'
, ref_key_2
from {src_table_name}
;
"""
print(insert_sql)


insert into g_relation (id
, uid
, ts
, rel_type
, ref_key
, ref_val
, ref_key_sub
, ref_val_sub)
select id
    , '0'
    , ts
    , 'team-person'
, 'g_person#url'
, ref_key
, 'g_person#url'
, ref_key_2
from t_person_team
;



In [87]:
with DBConn() as _conn:
    df_test2 = pd.read_sql(f"""
select id
    , '0'
    , ts
    , 'team-person'
, 'g_person#url'
, ref_key
, 'g_person#url'
, ref_key_2
from t_person_team
    """, _conn)
df_test2

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\2266003013.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test2 = pd.read_sql(f"""


,id,'0',ts,'team-person','g_person#url',ref_key,'g_person#url',ref_key_2
0,t_person # http://people.ece.cornell.edu/atang/,0,2023-04-16 10:01:01,team-person,g_person#url,http://networks.ece.cornell.edu/,g_person#url,http://people.ece.cornell.edu/atang/
1,t_person # https://yuchong-geng.github.io/,0,2023-04-16 10:01:01,team-person,g_person#url,http://networks.ece.cornell.edu/,g_person#url,https://yuchong-geng.github.io/
2,t_person # http://networks.ece.cornell.edu/faraz,0,2023-04-16 10:01:01,team-person,g_person#url,http://networks.ece.cornell.edu/,g_person#url,http://networks.ece.cornell.edu/faraz


In [88]:
# insert
with DBConn() as _conn:
    _conn.execute(insert_sql)
    _conn.commit()  

In [92]:
# query
with DBConn() as _conn:
    df_test3 = pd.read_sql(f"select * from {table_name} where rel_type='team-person';", _conn)
df_test3

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\2937123922.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test3 = pd.read_sql(f"select * from {table_name} where rel_type='team-person';", _conn)


,id,uid,ts,rel_type,ref_key,ref_val,ref_key_sub,ref_val_sub
0,t_person # http://people.ece.cornell.edu/atang/,0,2023-04-16 10:01:01,team-person,g_person#url,http://networks.ece.cornell.edu/,g_person#url,http://people.ece.cornell.edu/atang/
1,t_person # https://yuchong-geng.github.io/,0,2023-04-16 10:01:01,team-person,g_person#url,http://networks.ece.cornell.edu/,g_person#url,https://yuchong-geng.github.io/
2,t_person # http://networks.ece.cornell.edu/faraz,0,2023-04-16 10:01:01,team-person,g_person#url,http://networks.ece.cornell.edu/,g_person#url,http://networks.ece.cornell.edu/faraz


In [93]:
# data fix

update_sql = f"""
update {table_name}
set 
    ref_val = 'http://people.ece.cornell.edu/atang/'
where rel_type='team-person';
"""

with DBConn() as _conn:
    _conn.execute(update_sql)
    _conn.commit()  

In [97]:
# query
with DBConn() as _conn:
    df_test3 = pd.read_sql(f"""
        select * from {table_name} 
        where rel_type='team-person'
        and ref_val = ref_val_sub;
        """, _conn)
df_test3

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\2347093513.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test3 = pd.read_sql(f"""


,id,uid,ts,rel_type,ref_key,ref_val,ref_key_sub,ref_val_sub


In [96]:
# data fix

delete_sql = f"""
delete from {table_name} 
where rel_type='team-person'
and ref_val = ref_val_sub;
"""

with DBConn() as _conn:
    _conn.execute(delete_sql)
    _conn.commit()  

In [98]:
# query
with DBConn() as _conn:
    df_test3 = pd.read_sql(f"select * from {table_name} where rel_type='team-person';", _conn)
df_test3

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_17572\2937123922.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test3 = pd.read_sql(f"select * from {table_name} where rel_type='team-person';", _conn)


,id,uid,ts,rel_type,ref_key,ref_val,ref_key_sub,ref_val_sub
0,t_person # https://yuchong-geng.github.io/,0,2023-04-16 10:01:01,team-person,g_person#url,http://people.ece.cornell.edu/atang/,g_person#url,https://yuchong-geng.github.io/
1,t_person # http://networks.ece.cornell.edu/faraz,0,2023-04-16 10:01:01,team-person,g_person#url,http://people.ece.cornell.edu/atang/,g_person#url,http://networks.ece.cornell.edu/faraz
